In [ ]:
import os
import re
import pandas as pd 

def psentence(df):
    
    SENTENCE, PROBATION, FINE, APPEAL = list(), list(), list(), list()
    
    for i in range(len(df)):
        
        result = [str(), str(), str(), str()]
        try:
            data = df.loc[i, 'sentence'].replace(',', '')
        except:
            pass
        
        try:
            result[0] = re.findall(r'[징역|금고]\s(\d+년)?\s?(\d+?개?월)?', data)[0]
        except:
            pass
        
        if not any(result[0]):
            try:
                result[0] = re.findall(r'[징역|금고]\s(\d+년)\s?(\d+?개?월)?', data)[0]
            except:
                pass
        if not any(result[0]):
            try:
                result[0] = re.findall(r'[장기|단기]\s(\d+년)?\s?(\d+?개?월)?', data)[0]
            except:
                pass
        if not any(result[0]):
            try:
                result[0] = re.findall(r'[장기|단기]\s(\d+년)\s?(\d+?개?월)?', data)[0]
            except:
                pass
        if '무죄' in data:
            result[0] = '0'
        if '무기징역' in data:
            result[0] = '1000'
        if '사형' in data:
            result[0] = '1000'
        if type(result[0]) is tuple:
            if result[0][0] or result[0][1]:
                year = int(re.sub(r'\D', '', result[0][0]))*12  if result[0][0] else 0
                month = int(re.sub(r'\D', '', result[0][1])) if result[0][1] else 0
                result[0] = str(year+month)
            else:
                result[0] = str()
        
        result[0] = result[0] if result[0] and int(result[0]) <= 1000 else str()       
        
        result[1] = '1' if '유예' in data else str()
        
        try:
            result[2] = re.findall(r'[벌금\s]?([\d+,?.?]{1,}억?\s?[\d+,?.?]{1,}?만?\s?원)[^심고권]', data)[0]
            result[2] = str(int(re.findall(r'(\d+)억', result[2])[0])*100000000 + int(re.findall(r'(\d+)만', result[2])[0])*10000) if '억' in result[2] else result[2]
            result[2] = str(int(re.sub(r'\D', '', result[2]))*10000) if '만' in result[2] else re.sub(r'\D', '', result[2])
        except:
            pass
        result[2] = result[2] if result[2] and int(result[2]) >= 348 and '형사보상' not in data else str() 
        
        result[3] = '0' if '파기' in data else str()
        if result[3] == str():
            result[3] = '1' if '기각' and '사건' in data else str()
        if result[3] == str():
            result[3] = '1' if '기각' and '항소' in data else str()
        if result[3] == str():
            result[3] = '0' if '취소' in data else str()
        if result[3] == str():
            result[3] = '0' if '제청' in data else str()
        if result[3] == str():
            result[3] = '0' if '재심을 개시' in data else str()
        if result[3] == str():
            result[3] = '0' if '형사보상' in data else str()
        
        SENTENCE.append(result[0]);PROBATION.append(result[1]);FINE.append(result[2]);APPEAL.append(result[3])
        
    df['y_sentence'] = SENTENCE;df['y_probation'] = PROBATION;df['y_fine'] = FINE;df['y_appeal'] = APPEAL
    
    return df

In [ ]:
xname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#5_law'}.xlsx")
df = pd.read_excel(xname, index_col=0)

df2 = psentence(df)

df2 = df2[['seqnum', 'title', 'url', 'path', 'text', 'image', 'appeal', 'cid', 'case',  
           'body', 'law', 'sentence', 'y_sentence', 'y_probation', 'y_fine', 'y_appeal']]

for i in range(len(df2)):
    try:
        if int(df2.loc[i, 'y_sentence']) < 37 and int(df2.loc[i, 'y_sentence'])!=0:
            if df2.loc[i, 'y_probation'] != '1':
                df2.at[i, 'y_probation'] = '0'
    except:
        pass
    
df2.reset_index(drop=True, inplace=True)
x2name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#6_arrange'}.xlsx")
df2.to_excel(x2name)

In [ ]:
xname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#6_arrange'}.xlsx")
df = pd.read_excel(xname, index_col=0)

df = df.drop(columns=['seqnum', 'path', 'text', 'image', 'sentence'])
x2name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#7_dropped'}.xlsx")
df.to_excel(x2name)